In [1]:
using DataFrames
using CSV

# Purpose
This module is designed for reading in marker and dye data file.

The data shall be sotred in ".csv" format.

## Dyes
### dye_excitation.csv
spectral data of excitation of each dye, from 300 to 999 nm (step 1 nm).

### dye_emission.csv
spectral data of emission of each dye, from 300 to 999 nm (step 1 nm).

## Markers
### antigens.csv
first column as the symbol of each biomarker.

second column as each correlated antigen, 

thrid column as the relative brightness. \[level 1 to 5\]

In [5]:
laser_names = [:red, :yellow, :blue, :violet]

lasers = Dict(
    :red => 640,
    :yellow => 561,
    :blue => 488,
    :violet => 405
)

lens = Dict(
    :red    => [(660, 20), (730, 45), (780, 60)],
    :yellow => [(586, 15), (610, 20), (670, 30), (710, 50), (780, 60)],
    :blue   => [(530, 30), (575, 26), (610, 20), (670, 30), (710, 50), (780, 60)],
    :violet => [(450, 50), (510, 50), (605, 40), (660, 40)]
)

Dict{Symbol,Array{Tuple{Int64,Int64},1}} with 4 entries:
  :red    => Tuple{Int64,Int64}[(660, 20), (730, 45), (780, 60)]
  :yellow => Tuple{Int64,Int64}[(586, 15), (610, 20), (670, 30), (710, 50), (78…
  :violet => Tuple{Int64,Int64}[(450, 50), (510, 50), (605, 40), (660, 40)]
  :blue   => Tuple{Int64,Int64}[(530, 30), (575, 26), (610, 20), (670, 30), (71…

In [3]:
lasers

Dict{Symbol,Int64} with 4 entries:
  :red    => 640
  :yellow => 561
  :violet => 405
  :blue   => 488

In [17]:
excitation = CSV.read("../data/常见染料_发射曲线.csv",
    #types=[Int,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64],
    nullable=false
)

emission = CSV.read("../data/常见染料_激发曲线.csv",
    #types=[Int,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64],
    nullable=false
)

antigens = CSV.read("../data/使用抗体.csv",nullable=false)


,markers,dyes,brightness
1,Bcl_6,Alexa647,4
2,CD11c,Alexa647,4
3,GL_7,Alexa647,4
4,CD8alpha,Alexa700,1
5,B220,APC,5
6,CD3,APC,5
7,CD45_1,APC,5
8,CD45_2,APC,5
9,CD8,APC,5
10,CD80,APC,5


In [18]:
dyes_em = [item for item in emission.colindex.names if item != :Wavelength && item !=:wavelength] |> sort!
dyes_ex = [item for item in excitation.colindex.names if item != :Wavelength && item !=:wavelength] |> sort!
if dyes_em == dyes_ex
    dyes = dyes_ex
else
    println("names of dyes not match!")
    # TODO: using logging module
end
markers = [Symbol(item) for item in antigens[:markers]] |> unique

44-element Array{Symbol,1}:
 :Bcl_6     
 :CD11c     
 :GL_7      
 :CD8alpha  
 :B220      
 :CD3       
 :CD45_1    
 :CD45_2    
 :CD8       
 :CD80      
 :F4_80     
 :FAS       
 :IgG       
 ⋮          
 :MHCII     
 :TCR_beta  
 :Thy1_2    
 :AnnexinV  
 :CD38      
 :CD86      
 :CXCR4     
 :TCRValpha2
 :Thy1_1    
 :PD_1      
 :PDL_1     
 :CD205     

In [19]:
function getDyeBrightness(dye_list)
    default_chart = Dict()
    for item in dye_list
        default_chart[item] = 3
    end
    return chart-> begin
        for i =1:length(chart[:,1])
            name = Symbol(chart[i,1])
            value = chart[i,2]
            default_chart[name] = value
        end
        default_chart
    end
end
dye_brightness = antigens[2:3] |> unique |> getDyeBrightness(dyes)

Dict{Any,Any} with 15 entries:
  :APC_Cy7         => 2
  :Biotin_SA_APC   => 20
  :Alexa700        => 1
  :APC             => 5
  :Percp_Cy5_5     => 3
  :BV605           => 4
  :BV510           => 3
  :PE_Dazzle       => 5
  :Alexa647        => 4
  :Percp_cy5_5     => 2
  :PacificBlue     => 2
  :Biotin_SA_BV605 => 16
  :PE              => 5
  :PE_Cy7          => 4
  :FITC            => 2

In [28]:
function getExcitedRatio(lasers, dye_excitation)
    result = Dict()
    for (laser_name, laser_lambda) in lasers
        index = find(x->x==laser_lambda, dye_excitation[:Wavelength])[1]
        target = dye_excitation[index, :]
        result[laser_name] = target
    end
    result
end

function getLensNumber(lens)
    count = 0
    for (name, port) in lens
        count += length(port)
    end
    count
end

function getEmitRatio(lens, dye_emission)
    result = Dict()
    for (laser_name, ports) in lens
        port_result = Dict()
        for (target, range) in ports
            ranger = find(x->(target-range)<=x<=(target+range), emission[:Wavelength])
            port_result[target] = emission[ranger[1]:ranger[end],2:end]
        end
        result[laser_name] = port_result
    end
    result
end
            

function generateLensInteraction(emit_ratio, excite_ratio, dye_list, laser_list, lens, 
        dye_brightness)
    
    result = Dict()

    for (each_laser, _) in laser_list
        laser_array = Dict()
        for (port, _) in lens[each_laser]
            # println(each_laser, " ", port)
            port_array = Dict()
            for dominator in dye_list
                domi_value = excite_ratio[each_laser][dominator][1] / 100 * 
                             mean(emit_ratio[each_laser][port][dominator]) *
                             dye_brightness[dominator]
                dye_array = []
                for liberator in dye_list
                    if liberator != dominator
                        libe_value = excite_ratio[each_laser][liberator][1] / 100 * 
                                     mean(emit_ratio[each_laser][port][liberator]) *
                                     dye_brightness[liberator]
                        push!(dye_array, (domi_value-libe_value)/(domi_value+libe_value+1))
                    else
                        push!(dye_array, -Inf) 
                    end
                    
                end
                port_array[dominator] = dye_array
            end
            laser_array[port]=port_array
        end
        result[each_laser] = laser_array
    end
    result
end

generateLensInteraction (generic function with 1 method)

$$ \frac{\alpha_{ex}}{100} \cdot \langle \alpha_{em} \rangle \cdot \alpha_L$$

In [305]:
demo = generateLensInteraction(
    getEmitRatio(lens, emission, dyes),
    getExcitedRatio(lasers, excitation, dyes),
    dyes, lasers, lens, dye_brightness
)

Dict{Any,Any} with 4 entries:
  :red    => Dict{Any,Any}(Pair{Any,Any}(780, Dict{Any,Any}(Pair{Any,Any}(:APC_…
  :yellow => Dict{Any,Any}(Pair{Any,Any}(780, Dict{Any,Any}(Pair{Any,Any}(:APC_…
  :violet => Dict{Any,Any}(Pair{Any,Any}(510, Dict{Any,Any}(Pair{Any,Any}(:APC_…
  :blue   => Dict{Any,Any}(Pair{Any,Any}(780, Dict{Any,Any}(Pair{Any,Any}(:APC_…

In [306]:
demo[:red][780][:BV605]

12-element Array{Any,1}:
    0.0      
   -0.161327 
    0.0      
   -0.0374479
    0.0      
 -Inf        
    0.0      
    0.0      
   -0.848819 
    0.0      
    0.0      
    0.0      

In [31]:
getEmitRatio(lens, emission)[:red][780]

,Alexa647,Alexa700,APC,FITC,PacificBlue,PE,PE_Cy7,Percp_Cy5_5,BV510,PE_Dazzle,BV605,APC_Cy7
1,0.0,45.0411,0.0,0.0,0.0,0.0,20.5178239,0.0,0.0,0.0,0.0,21.116
2,0.0,42.15545,0.0,0.0,0.0,0.0,20.9617631,0.0,0.0,0.0,0.0,19.922
3,0.0,39.47933,0.0,0.0,0.0,0.0,21.5484197,0.0,0.0,0.0,0.0,18.765
4,0.0,36.80506,0.0,0.0,0.0,0.0,22.1258144,0.0,0.0,0.0,0.0,17.634
5,0.0,34.24891,0.0,0.0,0.0,0.0,22.7251008,0.0,0.0,0.0,0.0,16.52
6,0.0,31.74508,0.0,0.0,0.0,0.0,23.3683812,0.0,0.0,0.0,0.0,15.408
7,0.0,29.47728,0.0,0.0,0.0,0.0,24.1495374,0.0,0.0,0.0,0.0,14.283
8,0.0,27.28038,0.0,0.0,0.0,0.0,24.9149064,0.0,0.0,0.0,0.0,13.117
9,0.0,25.26911,0.0,0.0,0.0,0.0,25.6362813,0.0,0.0,0.0,0.0,11.891
10,0.0,23.30802,0.0,0.0,0.0,0.0,26.4031238,0.0,0.0,0.0,0.0,10.629


---

In [303]:
function getTemplate(antigens, marker_list, dye_list, level=1)
    template = zeros(length(marker_list), length(dye_list))
    for i=1:length(antigens[:,1])
        a1 = indexin([antigens[i,:markers] |> Symbol], marker_list)[1]
        a2 = indexin([antigens[i,:dyes] |> Symbol], dye_list)[1]

        if a2 == 0 
            continue
        end

        template[a1, a2] = 1
    end
    template
    repeat(template, outer=(1,1,level))
end
getTemplate(antigens, markers, dyes, getLensNumber(lens))

44×12×18 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮       
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 

In [15]:
lasers.count

4

In [7]:
?lasers

search:



No documentation found.

`lasers` is of type `Dict{Symbol,Int64}`.

**Summary:**

```
mutable struct Dict{Symbol,Int64} <: Associative{Symbol,Int64}
```

**Fields:**

```
slots    :: Array{UInt8,1}
keys     :: Array{Symbol,1}
vals     :: Array{Int64,1}
ndel     :: Int64
count    :: Int64
age      :: UInt64
idxfloor :: Int64
maxprobe :: Int64
```


In [22]:
typeof(emission)

DataFrames.DataFrame

In [23]:
markers

44-element Array{Symbol,1}:
 :Bcl_6     
 :CD11c     
 :GL_7      
 :CD8alpha  
 :B220      
 :CD3       
 :CD45_1    
 :CD45_2    
 :CD8       
 :CD80      
 :F4_80     
 :FAS       
 :IgG       
 ⋮          
 :MHCII     
 :TCR_beta  
 :Thy1_2    
 :AnnexinV  
 :CD38      
 :CD86      
 :CXCR4     
 :TCRValpha2
 :Thy1_1    
 :PD_1      
 :PDL_1     
 :CD205     